WEB SCRAPING

Parte 1: Entender el sitio objetivo
- Analizar la estructura de la página web

In [1]:
%pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from bs4 import BeautifulSoup

file_path = '../data/chicken.html'

with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')


In [ ]:

tittle= soup.find("meta", {"property": "og:title"})["content"]

In [4]:
tittle

'Rotisserie Chicken'

In [5]:
rating_div = soup.find("div", id="mm-recipes-review-bar__rating_1-0")
print('Rating: ',rating_div.text.strip())

Rating:  4.7


In [6]:
label = soup.find("div", class_="mm-recipes-details__label", string="Total Time:")
time_section = label.find_next_sibling("div", class_="mm-recipes-details__value")
print('Time: ', time_section.text.strip())

Time:  1 hr 30 mins


In [7]:
label = soup.find("div", class_="mm-recipes-details__label", string="Servings:")
serving_section = label.find_next_sibling("div", class_="mm-recipes-details__value")
print('Serving: ', serving_section.text.strip())


Serving:  6


In [8]:
ressume_class = soup.find("p", class_="article-subheading text-utility-300")
print('ressume: ',ressume_class.text.strip())

ressume:  This rotisserie chicken recipe is so easy to make with simple seasonings on your grill. Occasional basting with a butter mixture ensures crispy skin and moist meat. Our family loves this! Rotisserie chicken is perfect as the main dish with French fries and coleslaw, or with any number of other sides.


In [9]:
ingredient_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
for ingredient in ingredient_section:
    print("Ingredients: ",ingredient.text.strip())

Ingredients:  1 (3 pound) whole chicken
Ingredients:  1 pinch salt
Ingredients:  ¼ cup butter, melted
Ingredients:  1 tablespoon salt
Ingredients:  1 tablespoon ground paprika
Ingredients:  ¼ tablespoon ground black pepper


In [10]:
steps = soup.select("div#mm-recipes-steps__content_1-0 li")

print("Preparation Steps:")
for i, step in enumerate(steps, 1):
    paragraphs = step.find_all("p")
    step_text = " ".join(p.text.strip() for p in paragraphs if p.text.strip())
    print(f"Step {i}: {step_text}")

        

Preparation Steps:
Step 1: Gather all ingredients. Preheat an outdoor grill for high heat and lightly oil the grate. Dotdash Meredith Food Studios
Step 2: Season chicken cavity with a pinch of salt. Tie legs together with kitchen string; then tie wings to the bird. Secure chicken on a rotisserie attachment. Dotdash Meredith Food Studios
Step 3: Place rotisserie over the preheated grill and cook for 10 minutes. Dotdash Meredith Food Studios
Step 4: Meanwhile, quickly mix together butter, 1 tablespoon of salt, paprika, and pepper. Turn the grill down to medium and baste chicken with butter mixture. Dotdash Meredith Food Studios
Step 5: Close the lid and cook over medium heat, basting occasionally, until chicken is cooked through and the internal temperature reaches 180 degrees F (83 degrees C), 1 to 1 1/2 hours. Dotdash Meredith Food Studios
Step 6: Remove chicken from the rotisserie and let rest for 10 to 15 minutes before carving. Dotdash Meredith Food Studios


In [12]:
nutrition_table = soup.find("table", class_="mm-recipes-nutrition-facts-summary__table")

print("Nutrition Facts Summary:")
if nutrition_table:
    rows = nutrition_table.find_all("tr")
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 2:
            value = cols[0].text.strip()
            label = cols[1].text.strip()
            print(f"{label}: {value}")
else:
    print("No se encontró la tabla nutricional.")

Nutrition Facts Summary:
Calories: 357
Fat: 25g
Carbs: 1g
Protein: 31g


In [17]:
image = soup.find("img", id="mntl-sc-block-image_1-0")

if image:
    img_url = image.get("data-src") or image.get("src")
    print("Imagen principal:", img_url)

Imagen principal: https://www.allrecipes.com/thmb/ed37Lgu-NBfctOz7BNGWXLcfEo8=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/Screen-Shot-2022-06-08-at-11.14.27-AM-1ab22c6bdf894d82bbb9650e7da77e82.png


In [13]:
recipe_links = soup. find_all("a", href=True)

recipe_urls = []
for link in recipe_links:
    href = link[ 'href']
    if "recipe" in href:
        recipe_urls.append(href)

print("Linked Recipes:")
for url in recipe_urls:
    print(url)

Linked Recipes:
https://www.allrecipes.com/recipe/93168/rotisserie-chicken/#main
https://www.allrecipes.com/
https://www.allrecipes.com/recipes/17562/dinner/
https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/
https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/
https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/
https://www.allrecipes.com/recipes/17889/everyday-cooking/family-friendly/family-dinners/
https://www.allrecipes.com/recipes/94/soups-stews-and-chili/
https://www.allrecipes.com/recipes/16099/everyday-cooking/comfort-food/
https://www.allrecipes.com/recipes/80/main-dish/
https://www.allrecipes.com/recipes/22992/everyday-cooking/sheet-pan-dinners/
https://www.allrecipes.com/recipes/17562/dinner/
https://www.allrecipes.com/recipes-a-z-6735880
https://www.allrecipes.com/recipes/78/breakfast-and-brunch/
h

In [28]:
import json
import requests
import time
import re
from bs4 import BeautifulSoup

# --- Cabeceras simulando navegador ---
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

# --- Recolectar solo URLs válidas de receta ---
recipe_urls = []
pattern = re.compile(r"^https://www\.allrecipes\.com/(recipe)/\d+/")

for link in recipe_links:
    href = link.get('href')
    if href and pattern.match(href) and href not in recipe_urls:
        recipe_urls.append(href)



# --- Diccionario para almacenar los datos de todas las recetas ---
all_recipes = {}

# --- Iterar sobre cada URL de receta ---
for url in recipe_urls:
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # --- Título ---
        title_tag = soup.find("meta", {"property": "og:title"})
        title = title_tag["content"] if title_tag else None

        # --- Rating ---
        rating_div = soup.find("div", id="mm-recipes-review-bar__rating_1-0")
        rating = rating_div.text.strip() if rating_div else None

        # --- Tiempo total ---
        label_time = soup.find("div", class_="mm-recipes-details__label", string="Total Time:")
        total_time = label_time.find_next_sibling("div", class_="mm-recipes-details__value").text.strip() if label_time else None

        # --- Porciones ---
        label_servings = soup.find("div", class_="mm-recipes-details__label", string="Servings:")
        servings = label_servings.find_next_sibling("div", class_="mm-recipes-details__value").text.strip() if label_servings else None

        # --- Resumen ---
        resume_tag = soup.find("p", class_="article-subheading text-utility-300")
        resume = resume_tag.text.strip() if resume_tag else None

        # --- Ingredientes ---
        ingredients = [i.text.strip() for i in soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")]

        # --- Pasos ---
        steps = []
        step_tags = soup.select("div#mm-recipes-steps__content_1-0 li")
        for step in step_tags:
            paragraphs = step.find_all("p")
            step_text = " ".join(p.text.strip() for p in paragraphs if p.text.strip())
            if step_text:
                steps.append(step_text)

        # --- Información nutricional ---
        nutrition_table = soup.find("table", class_="mm-recipes-nutrition-facts-summary__table")
        nutrition_facts = {
            row.find_all("td")[1].text.strip(): row.find_all("td")[0].text.strip()
            for row in nutrition_table.find_all("tr")
            if len(row.find_all("td")) >= 2
        } if nutrition_table else {}

        # --- Imagen principal ---
        image = soup.find("img", id="mntl-sc-block-image_1-0")
        img_url = image.get("data-src") or image.get("src") if image else None

        # --- Guardar receta ---
        all_recipes[url] = {
            "title": title,
            "rating": rating,
            "total_time": total_time,
            "servings": servings,
            "resume": resume,
            "ingredients": ingredients,
            "steps": steps,
            "nutrition_facts": nutrition_facts,
            "image_url": img_url
        }

        print(f"✅ Receta procesada: {title}")

        time.sleep(1.5)  # Pausa entre requests para evitar bloqueo

    except Exception as e:
        print(f"❌ Error al procesar {url}: {e}")

# --- Guardar en archivo JSON ---
output_path = '../data/recipes_data.json'

with open(output_path, 'w', encoding='utf-8') as json_file:
    json.dump(all_recipes, json_file, ensure_ascii=False, indent=4)

print(f"\n📝 Datos de {len(all_recipes)} recetas guardados en '{output_path}'")


✅ Receta procesada: Rotisserie Chicken
✅ Receta procesada: Rotisserie Chicken
✅ Receta procesada: Cilantro-Lime Grilled Chicken
✅ Receta procesada: Buttermilk Barbecue Chicken
✅ Receta procesada: Grilled Spatchcocked Chicken
✅ Receta procesada: Beer Butt Chicken
✅ Receta procesada: Good Frickin' Paprika Chicken
✅ Receta procesada: Miso Honey Chicken
✅ Receta procesada: Rosemary Buttermilk Chicken
✅ Receta procesada: Smoked Beer Butt Chicken
✅ Receta procesada: The Best Beer Can Chicken Ever
✅ Receta procesada: Best Beer Can Chicken
✅ Receta procesada: Drunk Chicken
✅ Receta procesada: Grilled Chicken Under a Brick
✅ Receta procesada: Smoked Whole Chicken
✅ Receta procesada: Easy Barbeque Chicken
✅ Receta procesada: Darn Good Chicken
✅ Receta procesada: Beer Can Chicken

📝 Datos de 18 recetas guardados en '../data/recipes_data.json'
